In [ ]:
# ============================================================================
# CELL 0: QUICK START - HARDCODED API KEY (OPTIONAL)
# ============================================================================
# WARNING: Only use this for LOCAL TESTING
# DO NOT push this cell to GitHub with your real API key!
# For production, use Cell 1 below with environment variables
# ============================================================================

# Uncomment these lines to hardcode your API key:

# import os
# import asyncio
# import requests
# from google.adk.agents import LlmAgent
# from google.adk.tools import AgentTool
# from google.adk.models.google_llm import Gemini
# from google.adk.runners import InMemoryRunner
# from google.adk.code_executors import BuiltInCodeExecutor
# from google.genai import types

# # Set your Google API key here
# GOOGLE_API_KEY = "AIzaSy_YOUR_KEY_HERE"  # Replace with your actual key

# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# print(" Setup complete with hardcoded API key")
# print(" Remember: DO NOT push this to GitHub!")

# ============================================================================
# After testing, comment out this cell and use Cell 1 (environment variable)
# ============================================================================


In [ ]:
# Imports and Setup
import os
import asyncio
import requests
from dotenv import load_dotenv  # THIS IS REQUIRED
from google.adk.agents import LlmAgent
from google.adk.tools import AgentTool
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types

# Load .env file ← THIS READS YOUR .env FILE
load_dotenv()

# Get API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError(
        "GOOGLE_API_KEY not found!"
        "Create .env file with: GOOGLE_API_KEY=your_key_here"
    )

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print(" Setup complete. API key loaded from .env file.")
print(f" Key preview: {GOOGLE_API_KEY[:20]}...")

In [ ]:
# Configuration
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=2,
    initial_delay=2,
    http_status_codes=[429, 500, 503, 504]
)

# Model configuration
# Options:
#   - gemini-1.5-flash: Recommended (higher free tier: 15 RPM, 1M TPM)
#   - gemini-2.5-flash-lite: Lower limits (20 req/day) but newer
#   - gemini-2.0-flash-exp: Experimental, may be unstable
model_config = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config
)

print("Model configuration created")

In [ ]:
# Custom Tools

# Simple cache dictionary
_rate_cache = {}
_cache_duration_minutes = 60  # Cache for 1 hour

def parse_currency(code_or_name: str) -> str:
    """Convert currency names to ISO codes."""
    mapping = {
        'usd': 'USD', 'dollar': 'USD', 'us dollar': 'USD', '$': 'USD',
        'eur': 'EUR', 'euro': 'EUR', 'euros': 'EUR', '€': 'EUR',
        'gbp': 'GBP', 'pound': 'GBP', 'pounds': 'GBP', '£': 'GBP',
        'try': 'TRY', 'lira': 'TRY', 'turkish lira': 'TRY', 'turkish': 'TRY',
        'cad': 'CAD', 'canadian dollar': 'CAD', 'canadian': 'CAD',
        'aud': 'AUD', 'australian dollar': 'AUD', 'australian': 'AUD',
    }
    return mapping.get(code_or_name.lower(), code_or_name.upper())

def get_fee_for_payment_method(from_currency: str, to_currency: str, amount: float) -> dict:
    from_currency = parse_currency(from_currency)
    to_currency = parse_currency(to_currency)

def get_fee_for_payment_method(from_currency: str, to_currency: str, amount: float) -> dict:
    """
    Returns transaction fee structure in SOURCE currency.
    """
    fee_structure = {
        ("USD", "EUR"): {"percentage": 0.02, "fixed_fee": 1.50},
        ("USD", "GBP"): {"percentage": 0.025, "fixed_fee": 2.00},
        ("USD", "TRY"): {"percentage": 0.02, "fixed_fee": 5.00},
        ("EUR", "USD"): {"percentage": 0.02, "fixed_fee": 1.50},
        ("EUR", "GBP"): {"percentage": 0.02, "fixed_fee": 1.50},
        ("EUR", "TRY"): {"percentage": 0.025, "fixed_fee": 8.00},
        ("GBP", "USD"): {"percentage": 0.025, "fixed_fee": 2.00},
        ("GBP", "EUR"): {"percentage": 0.02, "fixed_fee": 1.50},
        ("TRY", "USD"): {"percentage": 0.020, "fixed_fee": 50.0},
        ("TRY", "EUR"): {"percentage": 0.020, "fixed_fee": 50.0},
        ("USD", "CAD"): {"percentage": 0.015, "fixed_fee": 2.00},
        ("USD", "AUD"): {"percentage": 0.015, "fixed_fee": 2.50},
        ("CAD", "USD"): {"percentage": 0.015, "fixed_fee": 2.00},
        ("AUD", "USD"): {"percentage": 0.015, "fixed_fee": 2.50},
        ("CAD", "EUR"): {"percentage": 0.02, "fixed_fee": 2.50},
        ("CAD", "GBP"): {"percentage": 0.025, "fixed_fee": 3.00},
        ("CAD", "TRY"): {"percentage": 0.03, "fixed_fee": 10.00},
        ("CAD", "AUD"): {"percentage": 0.02, "fixed_fee": 2.75},
        ("AUD", "EUR"): {"percentage": 0.02, "fixed_fee": 3.00},
        ("AUD", "GBP"): {"percentage": 0.025, "fixed_fee": 3.50},
        ("AUD", "TRY"): {"percentage": 0.03, "fixed_fee": 15.00},
        ("AUD", "CAD"): {"percentage": 0.02, "fixed_fee": 2.75},
    }
    
    if (from_currency, to_currency) not in fee_structure:
        return {
            "status": "error",
            "message": f"Unsupported currency pair: {from_currency} → {to_currency}"
        }
    
    fee_info = fee_structure[(from_currency, to_currency)]
    pct_fee = amount * fee_info["percentage"]
    total = pct_fee + fee_info["fixed_fee"]
    
    return {
        "status": "success",
        "from_currency": from_currency,
        "to_currency": to_currency,
        "fee_currency": from_currency,
        "percentage_rate": fee_info["percentage"],
        "percentage_fee_amount": pct_fee,
        "fixed_fee": fee_info["fixed_fee"],
        "total_fee": total,
        "amount": amount,  # ← ADDED THIS KEY TO FIX THE ERROR
        "original_amount": amount,
        "amount_after_fees": amount - total
    }


def get_exchange_rate(from_currency: str, to_currency: str) -> dict:
    """
    Gets LIVE exchange rate from Frankfurter API with 1-hour caching.
    """
    from datetime import datetime, timedelta
    
    cache_key = f"{from_currency}_{to_currency}"
    now = datetime.now()
    
    # Check if we have valid cached data
    if cache_key in _rate_cache:
        cached_time, cached_data = _rate_cache[cache_key]
        age_minutes = (now - cached_time).total_seconds() / 60
        
        # If cache is still valid (less than 60 minutes old)
        if age_minutes < _cache_duration_minutes:
            print(f"✅ Using cached rate (age: {int(age_minutes)} min)")
            return cached_data
    
    # Cache expired or doesn't exist - fetch fresh data
    print(f"🔄 Fetching fresh rate from API...")
    
    try:
        url = f"https://api.frankfurter.app/latest?from={from_currency}&to={to_currency}"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        
        data = response.json()
        
        if "rates" not in data or to_currency not in data["rates"]:
            return {
                "status": "error",
                "message": f"Currency pair not available: {from_currency} → {to_currency}"
            }
        
        result = {
            "status": "success",
            "from_currency": from_currency,
            "to_currency": to_currency,
            "exchange_rate": data["rates"][to_currency],
            "timestamp": data.get("date"),
            "source": "Frankfurter API"
        }
        
        # Save to cache
        _rate_cache[cache_key] = (now, result)
        print(f" Rate cached for {_cache_duration_minutes} minutes")
        
        return result
    
    except Exception as e:
        # If API fails and we have old cache, use it
        if cache_key in _rate_cache:
            print(f"⚠️ API failed, using old cache")
            return _rate_cache[cache_key][1]
        
        return {
            "status": "error",
            "message": f"API error: {str(e)}"
        }


print(" Custom tools created (with 1-hour caching)")

In [ ]:
# Calculation Agent
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=model_config,
    instruction="""You are a specialized calculator that ONLY responds with Python code.

Your task is to take a request for a calculation and translate it into Python code.

RULES:
1. Output ONLY Python code, no explanations
2. The code must calculate and print the result
3. Always define variables before using them
4. Use simple print statements with string concatenation

EXAMPLE:
If asked "Calculate: 5000 minus 200 fee gives 4800, then multiply by 0.92":

original_amount = 5000
fee_amount = 200.0
after_fee_amount = 4800.0
exchange_rate = 0.92
final_amount = after_fee_amount * exchange_rate

print("Original Amount:", original_amount)
print("Fee:", fee_amount)
print("After Fees:", after_fee_amount)
print("Exchange Rate:", exchange_rate)
print("Final Amount:", round(final_amount, 2))

Replace the numbers with the actual values from the request.
""",
    code_executor=BuiltInCodeExecutor()
)

print(" Calculation Agent created with BuiltInCodeExecutor")

In [ ]:
# Enhanced Currency Agent with Better Output Formatting
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=model_config,
    instruction="""You are a currency conversion assistant that coordinates multiple tools.

SUPPORTED CURRENCIES:
USD (Dollar), EUR (Euro), GBP (Pound), TRY (Turkish Lira), CAD (Canadian Dollar), AUD (Australian Dollar)

WORKFLOW:
1. Call get_fee_for_payment_method(from_currency, to_currency, amount) to get fee information
2. Call get_exchange_rate(from_currency, to_currency) to get the live exchange rate
3. Use the CalculationAgent to perform the final calculation

UNSUPPORTED HANDLING:
If currency not in USD,EUR,GBP,TRY,CAD,AUD, respond:
"Sorry, I support USD, EUR, GBP, TRY, CAD, AUD only"

IMPORTANT RULES FOR CALLING CalculationAgent:
- Extract the actual numeric values from the tool responses
- Pass these numbers directly in your request to CalculationAgent
- DO NOT use variable names or placeholders
- Spell out the calculation with actual numbers

OUTPUT FORMAT - Present results in this clear format:

CONVERSION SUMMARY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Original Amount: [amount] [from_currency]
Transaction Fee: [fee] [from_currency]
Amount After Fees: [after_fees] [from_currency]
Exchange Rate: 1 [from_currency] = [rate] [to_currency]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Final Amount: [result] [to_currency]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Replace the bracketed values with actual numbers from your calculations.
""",
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),
    ],
)

print(" Enhanced Currency Agent created with better output formatting")

In [ ]:
# Test Function with Better Formatted Output
async def test_conversion(agent, query: str, agent_name: str = "Enhanced Agent"):
    """
    Test currency conversion agent with comprehensive error handling
    and nicely formatted output.
    """
    try:
        # Header
        print("\n" + "=" * 60)
        print("CURRENCY CONVERSION TEST")
        print("=" * 60)
        print(f"Agent: {agent_name}")
        print(f"Query: {query}")
        print("-" * 60)
        
        runner = InMemoryRunner(agent=agent)
        
        # Capture and display events
        print("\nProcessing...\n")
        events = await runner.run_debug(query)
        
        # Success footer
        print("\n" + "-" * 60)
        print(" CONVERSION COMPLETED SUCCESSFULLY")
        print(" Data source: Live exchange rates from Frankfurter API")
        print("=" * 60 + "\n")
        
    except KeyError as ke:
        print("\n" + "!" * 60)
        print(f" ERROR: KeyError - {ke}")
        print("!" * 60)
        print("\n💡 Troubleshooting:")
        print("   • The agent tried to access a missing dictionary key")
        print("   • Check that tool functions return all expected keys")
        print("   • Run the DEBUG cell to verify tool output")
        
    except Exception as e:
        print("\n" + "!" * 60)
        print(f" ERROR: {type(e).__name__}")
        print("!" * 60)
        error_msg = str(e)
        
        if "RESOURCE_EXHAUSTED" in error_msg:
            print("\n API Rate Limit Hit!")
            print("   Solutions:")
            print("   • Wait 1 minute for rate limit reset")
            print("   • Switch to gemini-1.5-flash (higher limit)")
            print("   • Get paid API key for unlimited access")
        else:
            print(f"\n Details: {error_msg[:250]}")

print(" Test function ready with improved formatting")

In [ ]:
# Run Currency Conversion Tests

# Single test
print(" Currency Conversion with LIVE API ")

await test_conversion(
    enhanced_currency_agent,
    "Convert 1000 EUR to TRY with fees",
    "Enhanced Agent"
)